In [3]:
from scipy.optimize import minimize
import numpy as np
from random import sample
from numba import jit
from time import time

@jit
def LJ(r):
    r6 = r**6
    r12 = r6*r6
    return 4*(1/r12 - 1/r6)

@jit
def total_energy(points): #points is a 3*N array of all the atoms involved
    total = 0
    num_atoms = int(len(points)/3)
    
    for i in range(num_atoms - 1):
        for j in range(i+1, num_atoms):
            pos1 = points[i*3 : (i+1)*3]
            pos2 = points[j*3 : (j+1)*3]
            
            r = np.linalg.norm(pos1-pos2)
            
            total += LJ(r)
    return total

@jit
def gradient(pos):
    """
    Calculate and return the Jacobian for minimizing
    the Lennard-Jones potential between N atoms.
    Input:
        positions: 3*N 1-D array of x,y,z positions of each atom
    Output:
        3*N 1-D array of the Jacobian elements for N atoms.
    """
    N = int(len(pos)/3)
    J = np.zeros([N, 3]) 
    
    for i in range(N-1):
        center = pos[3*i:(3*i)+3]
        for j in range(i+1, N):
            neighbor = pos[3*j:(3*j)+3] 
            r = np.linalg.norm(center - neighbor)
            J[i] += (24/r**7 - 48/r**13)*(center - neighbor)/r
    
    return J.flatten()
def init_pos(N, L=5):
    return L*np.random.random_sample((N*3,))

def init_population(population_size, N_atom):
    cluster = []
    for i in range(population_size):
        cluster.append(init_pos(N_atom))
    return np.array(cluster)

def local_optimize(cluster):
    fitness = []
    for i, cluster0 in enumerate(cluster):
        res = minimize(total_energy, cluster0, method='Powell', tol=1e-4, options={'maxiter':20}) #opt w/o gradient
        res = minimize(total_energy, res.x, method='CG', jac=gradient, tol=1e-4) #analytical gradient
        #res = minimize(total_energy, cluster0, method='CG', tol=1e-3) 
        cluster0 = res.x
        fitness.append(res.fun)
    return cluster, np.array(fitness)

def selTournament(fitness, factor=0.35):
    """
    Select the best individual among *tournsize* randomly chosen
    individuals, *k* times. The list returned contains
    references to the input *individuals*.
    """
    IDs = sample(set(range(len(fitness))), int(len(fitness)*factor))
    min_fit = np.argmin(fitness[IDs])
    return IDs[min_fit]

def ranking(fitness):
    return np.argsort(fitness)

def mutation(cluster, fitness, kT=0.5):
    """
    Select a structure and perturb it
    """
    id = selTournament(fitness)
    cluster0 = cluster[id]
    disp = np.random.random_sample((len(cluster0),))-0.5
                                   
    return cluster0 + kT*disp

def crossover(cluster, fitness):
    """
    Select two structures and do sort of *crossover*
    """
    id1 = selTournament(fitness)
    while True:
        id2 = selTournament(fitness)
        if id2 != id1:
            break
    frac = np.random.random()
    #todo: center the cluster, and then xA+(1-x)B 
    return cluster[id1]*frac + cluster[id2]*(1-frac)

def GA(N=10, gen=10, pop=10, ratio=0.7):
    """
    The main GA code,
    
    Args:
        - N: number of atoms
        - gen: number of generations
        - pop: number of populations
        - ratio: ratio of crossover
    """
    for i in range(gen):
        t0 = time()
        if i == 0:
            cluster = init_population(pop, N)
            
        cluster, fitness = local_optimize(cluster)
        #print(fitness)
        print('best fitness in generation ', i, ':  ', min(fitness), ' time:', time()-t0)
        
        #rank = ranking(fitness)
        #selecton process
        new_cluster = []
        for j in range(pop):
            if j < int(ratio*pop):
                new_cluster.append(crossover(cluster, fitness))
            else:
                new_cluster.append(mutation(cluster, fitness))
        cluster = new_cluster

In [4]:
N_atom = 12

In [8]:
from scipy.optimize import differential_evolution


pos = init_pos(N_atom)
popl = init_population(10, N_atom)

bound = []
seq = []
for j in range(len(pos)):
    seq = (-5, 5)
    bound.append(seq)

res = differential_evolution(total_energy, bounds=bound, maxiter=50, disp=True)

differential_evolution step 1: f(x)= -3.60662
differential_evolution step 2: f(x)= -3.71457
differential_evolution step 3: f(x)= -3.71457
differential_evolution step 4: f(x)= -3.71457
differential_evolution step 5: f(x)= -3.97095
differential_evolution step 6: f(x)= -3.97095
differential_evolution step 7: f(x)= -3.97095
differential_evolution step 8: f(x)= -3.97095
differential_evolution step 9: f(x)= -3.97095
differential_evolution step 10: f(x)= -3.97095
differential_evolution step 11: f(x)= -4.93561
differential_evolution step 12: f(x)= -4.93561
differential_evolution step 13: f(x)= -4.93561
differential_evolution step 14: f(x)= -4.93561
differential_evolution step 15: f(x)= -4.93561
differential_evolution step 16: f(x)= -4.93561
differential_evolution step 17: f(x)= -4.93561
differential_evolution step 18: f(x)= -4.93561
differential_evolution step 19: f(x)= -4.93561
differential_evolution step 20: f(x)= -4.93561
differential_evolution step 21: f(x)= -4.93561
differential_evolution